# 1. Installation and Basic of TVM

### [About TVM](https://tvm.ai/about)

TVM is an open deep learning compiler stack for CPUs, GPUs, and specialized accelerators. It aims to close the gap between the productivity-focused deep learning frameworks, and the performance- or efficiency-oriented hardware backends. TVM provides the following main features:

- Compilation of deep learning models in Keras, MXNet, PyTorch, Tensorflow, CoreML, DarkNet into minimum deployable modules on diverse hardware backends.
- Infrastructure to automatic generate and optimize tensor operators on more backend with better performance.

![image.png](https://tvm.ai/images/main/tvm-stack.png)

### [Background](https://github.com/apache/incubator-mxnet/issues/15465)

Currently in MXNet we implement operator kernels in C++. Developers need to specify the detail logic of each computation, which slows down the development process. Given the fact that we’re moving forward to be numpy-compatible, a large amount of operators are to be implemented. Moreover, we also have various of backend to support, including CPUs and GPUs from AMD, ARM, Intel, Nvidia, etc. It requires great effort to implement efficient kernels for each of these hardwares, so as writing test cases for each of the operator+backend combination.

## 1.1 Installation

Official Guide: https://docs.tvm.ai/install/from_source.html

1. **Install python environment**
2. **Download source code**
```
git clone --recursive https://github.com/dmlc/tvm
```
3. **Install backend**
 - Create build directory
```
cd tvm
mkdir build
cp cmake/config.cmake build
cd build
```
 - Download llvm.
```
wget http://releases.llvm.org/8.0.0/clang+llvm-8.0.0-x86_64-linux-gnu-ubuntu-16.04.tar.xz
tar -xf clang+llvm-8.0.0-x86_64-linux-gnu-ubuntu-16.04.tar.xz
mv clang+llvm-8.0.0-x86_64-linux-gnu-ubuntu-16.04 llvm
```
 - Edit file `config.cmake`. Change `set(USE_CUDA OFF)` to `set(USE_CUDA ON)`, and change `set(USE_LLVM OFF)` to `set(USE_LLVM /home/ubuntu/tvm/build/llvm/bin/llvm-config)`
 - Then build tvm
```
cd build
cmake ..
make -j4
```
4. **Install python frontend**
```
cd ..
cd python && pip install -e . && cd ..
cd topi/python && pip install -e . && cd ../..
cd nnvm/python && pip install -e . && cd ../..
```

## 1.2 Enable TVMOp in MXNet

**CMake:** -DUSE_TVM_OP=ON

**Make:** USE_TVM_OP = 1

## 1.3 Basic Usage

TVM uses symbolic API like tensorflow, it just defines the computation flow. In TVM, every node is a `Tensor` and has its operator, contains `placeholder`(its output same as input).

Especially, when we use tvm to implement an operator, we don't know any imformation of **shape**. But we know **ndim** and **dtypes**, actually the kernels are generated for each ndim/dtype.

We use a simple vector add as example.

In [2]:
import tvm

A = tvm.placeholder([tvm.var() for _ in range(1)], name='A', dtype="float32")
B = tvm.placeholder([tvm.var() for _ in range(1)], name='B', dtype="float32")
C = tvm.compute([tvm.var() for _ in range(1)], lambda *i: A[i] + B[i], name='C')
# C = tvm.compute([tvm.var() for _ in range(2)], lambda *i: A(*i) + B(*i), name='C')
# C = tvm.compute([tvm.var() for _ in range(2)], lambda i, j: A[i, j] + B[i, j], name='C')
s = tvm.create_schedule(C.op)

In [3]:
type(A)

tvm.tensor.Tensor

In [4]:
C.dtype

'float32'

In [5]:
type(A.op)

tvm.tensor.PlaceholderOp

In [6]:
type(s), type(s[C])

(tvm.schedule.Schedule, tvm.schedule.Stage)

In [7]:
pseudocode = tvm.lower(s, [A, B, C], simple_mode=True)
mod_cpu = tvm.build(s, [A, B, C], target='llvm')
print(pseudocode)
print('#' * 80)
print(mod_cpu.get_source())

produce C {
  for (i0, 0, tindex) {
    C[(i0*stride)] = (A[(i0*stride)] + B[(i0*stride)])
  }
}

################################################################################
; ModuleID = 'default_function'
source_filename = "default_function"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

%0 = type { i8*, %1, i32, %2, i64*, i64*, i64 }
%1 = type { i32, i32 }
%2 = type { i8, i8, i16 }

@__TVMAPISetLastError = linkonce dllexport local_unnamed_addr global void (i8*)* null, align 8
@.str = private constant [69 x i8] c"Assert fail: (num_args == 3), default_function: num_args should be 3\00", align 1
@.str.1 = private constant [144 x i8] c"Assert fail: ((((arg0.code == 3) || (arg0.code == 13)) || (arg0.code == 7)) || (arg0.code == 4)), default_function: Expect arg[0] to be pointer\00", align 1
@.str.2 = private constant [144 x i8] c"Assert fail: ((((arg1.code == 3) || (arg1.code == 13)) || (arg1.code == 7)) || (arg1.code == 4)), d

For we use contiguous tensor, without broadcast, the stride can be computed by reverse exclusive scan using multiplication. For example, the stride of a ndarray with shape **(2, 3, 4)** is **(12, 4, 1)**.

In [8]:
import numpy as np

a = tvm.nd.array(np.ones((3,), dtype="float32"))
b = tvm.nd.array(np.ones((1,), dtype="float32") * 2)
c = tvm.nd.array(np.zeros((3,), dtype="float32"))
mod_cpu(a, b, c)
c

<tvm.NDArray shape=(3,), cpu(0)>
array([3., 3., 3.], dtype=float32)

In [10]:
bx, tx = s[C].split(C.op.axis[0], factor=64)
s[C].bind(bx, tvm.thread_axis("blockIdx.x"))
s[C].bind(tx, tvm.thread_axis("threadIdx.x"))

pseudocode = tvm.lower(s, [A, B, C], simple_mode=True)
mod_gpu = tvm.build(s, [A, B, C], target='cuda')
print(pseudocode)
print('#' * 80)
print(mod_gpu.imported_modules[0].get_source())

produce C {
  // attr [iter_var(blockIdx.x, , blockIdx.x)] thread_extent = floordiv((tindex + 63), 64)
  // attr [iter_var(threadIdx.x, , threadIdx.x)] thread_extent = 64
  if (likely((((blockIdx.x*64) + threadIdx.x) < tindex))) {
    if (likely((((blockIdx.x*64) + threadIdx.x) < tindex))) {
      C[(((blockIdx.x*64) + threadIdx.x)*stride)] = (A[(((blockIdx.x*64) + threadIdx.x)*stride)] + B[(((blockIdx.x*64) + threadIdx.x)*stride)])
    }
  }
}

################################################################################
extern "C" __global__ void default_function_kernel0( float* __restrict__ C,  float* __restrict__ A,  float* __restrict__ B, int tindex, int stride, int stride1, int stride2) {
  if (((int)blockIdx.x) < (tindex >> 6)) {
    C[(((((int)blockIdx.x) * 64) + ((int)threadIdx.x)) * stride2)] = (A[(((((int)blockIdx.x) * 64) + ((int)threadIdx.x)) * stride)] + B[(((((int)blockIdx.x) * 64) + ((int)threadIdx.x)) * stride1)]);
  } else {
    if (((((int)blockIdx.x) * 64) + ((in

In [12]:
ctx = tvm.context('cuda', 0)
a = tvm.nd.array(np.ones((2,), dtype="float32"), ctx=ctx)
b = tvm.nd.array(np.ones((2,), dtype="float32") * 2, ctx=ctx)
c = tvm.nd.array(np.zeros((2,), dtype="float32"), ctx=ctx)
mod_gpu(a, b, c)
c

<tvm.NDArray shape=(2,), gpu(0)>
array([3., 3.], dtype=float32)

## 1.4 Advanced Usage

Do not support native python control flow, use `tvm.if_then_else` instead. Or try to use [hybrid script](https://docs.tvm.ai/langref/hybrid_script.html).

How about broadcast?

- https://docs.tvm.ai/api/python/tvm.html?highlight=auto_broadcast#tvm.decl_buffer

How to optimize the schedule of an operator? Refer to:

- https://docs.tvm.ai/tutorials/index.html#tensor-expression-and-schedules
- https://docs.tvm.ai/tutorials/language/schedule_primitives.html

Reshape, slicing, etc. See:

- http://tvm.d2l.ai

# 2. Use TVM to implement operator in MXNet backend

Also use `add` as example.

## Use python implement core compute function

**Core compute funtion, like kernel(s) in `.h` files:**

- Inputs: arguments that need traversion is necessary
- Outouts: schedule, input(s), output(s)

```python
# contrib/tvmop/basic/ufunc.py
def compute_add(atype, btype, ndim):
    A = tvm.placeholder([tvm.var() for _ in range(ndim)], name='A', dtype=atype)
    B = tvm.placeholder([tvm.var() for _ in range(ndim)], name='B', dtype=btype)
    C = tvm.compute([tvm.var() for _ in range(ndim)],
                    lambda *index: A[index].astype() + B[index].astype(), name='C')
    s = tvm.create_schedule(C.op)
    return s, A, B, C
```

**Device related compute function, similar to `FCompute`:**

- Function signature similar to above
- In decorator: `name`, `target`, `dtype` and `ndim` are necessary

```python
# contrib/tvmop/basic/ufunc.py
@defop(name="vadd", target="cpu", auto_broadcast=True,
       atypes=AllTypes, btypes=FloatTypes, ndim=[5])
def vadd(atype, btype, ndim):
    s, A, B, C = compute_add(dtype, ndim, '-')
    axes = [axis for axis in C.op.axis]
    fused = s[C].fuse(*axes)
    s[C].parallel(fused)

    return s, [A, B, C]


@defop(name="cuda_vadd", target="cuda", auto_broadcast=True,
       dtype=["float32", "float64"], ndim=[5])
def vadd_gpu(dtype, ndim):
    s, A, B, C = compute_add(dtype, ndim)
    s = tvm.create_schedule(C.op)
    axes = [axis for axis in C.op.axis]
    fused = s[C].fuse(*axes)
    bx, tx = s[C].split(fused, factor=64)
    s[C].bind(bx, tvm.thread_axis("blockIdx.x"))
    s[C].bind(tx, tvm.thread_axis("threadIdx.x"))
    return s, [A, B, C]
```

**Backward is similar:**

```python
# contrib/tvmop/basic/ufunc.py
def compute_backward_vadd(dtype, ndim, reduce1st, req):
    axes = ([reduce1st, 1 - reduce1st] * ndim)[:ndim]
    X = tvm.placeholder([tvm.var() for _ in range(ndim)], name='X', dtype=dtype)
    reducer = tvm.comm_reducer(lambda x, y: x + y,
        lambda t: tvm.const(0, dtype=t), name="sum")
    ret = reduce_axes(X, axes, reducer)
    in_grad_a, in_grad = assign_by_req(ret, req)
    s = tvm.create_schedule(in_grad.op)
    return s, X, in_grad_a, in_grad, [ret, in_grad]


@defop(name="backward_vadd", target="cpu", dtype=AllTypes,
       ndim=[5], reduce1st=[0, 1],
       req=["kWriteTo", "kAddTo"], attrs=["reduce1st", "req"])
def backward_vadd(dtype, ndim, reduce1st, req):
    s, X, in_grad_a, in_grad, c_list = compute_backward_vadd(dtype, ndim, reduce1st, req)
    for t in c_list:
        axes = [axis for axis in t.op.axis]
        fused = s[t].fuse(*axes)
        s[t].parallel(fused)
    return s, [X, in_grad_a, in_grad]


@defop(name="cuda_backward_vadd", target="gpu", dtype=["float32", "float64"],
       ndim=[5], reduce1st=[0, 1],
       req=["kWriteTo", "kAddTo"], attrs=["reduce1st", "req"])
def backward_vadd_gpu(dtype, ndim, reduce1st, req):
    s, X, in_grad_a, in_grad, c_list = compute_backward_vadd(dtype, ndim, reduce1st, req)
    num_thread = 64
    for t in c_list:
        block_x = tvm.thread_axis("blockIdx.x")
        thread_x = tvm.thread_axis("threadIdx.x")
        axes = [axis for axis in t.op.axis]
        fused = s[t].fuse(*axes)
        bx, tx = s[t].split(fused, factor=num_thread)
        s[t].bind(bx, block_x)
        s[t].bind(tx, thread_x)
    return s, [X, in_grad_a, in_grad]
```

## Register operator in backend

```c++
// src/operator/contrib/tvmop/ufunc.cc
static constexpr char func_vadd_cpu[] = "vadd";
static constexpr char func_vadd_gpu[] = "cuda_vadd";
static constexpr char func_bakcward_vadd_cpu[] = "backward_vadd";
static constexpr char func_bakcward_vadd_gpu[] = "cuda_backward_vadd";

template<const char* func>
void TVMBinaryCompute(const nnvm::NodeAttrs& attrs,
                      const mxnet::OpContext& ctx,
                      const std::vector<TBlob>& inputs,
                      const std::vector<OpReqType>& req,
                      const std::vector<TBlob>& outputs) {
  CHECK_EQ(inputs.size(), 2U);
  CHECK_EQ(outputs.size(), 1U);
  TBlob idata[2], odata;
  for (int k = 0; k < 2; ++k) {
    idata[k] = Padding(inputs[k], max_dim);
  }
  odata = Padding(outputs[0], max_dim);
  tvm::runtime::TVMOpModule::Get()->Call(func, ctx, {idata[0], idata[1], odata});
}

NNVM_REGISTER_OP(_contrib_tvm_vadd)
    .set_num_inputs(2)
    .set_num_outputs(1)
    .add_argument("a", "NDArray-or-Symbol", "first input")
    .add_argument("b", "NDArray-or-Symbol", "second input")
    .set_attr<nnvm::FListInputNames>("FListInputNames",
      [](const NodeAttrs& attrs) {
        return std::vector<std::string>{"a", "b"};
      })
    .set_attr<mxnet::FInferShape>("FInferShape", BinaryBroadcastShape)
    .set_attr<nnvm::FInferType>("FInferType", mxnet::op::ElemwiseType<2, 1>)
#if MXNET_USE_CUDA
    .set_attr<mxnet::FCompute>("FCompute<gpu>", mxnet::op::TVMBinaryCompute<func_vadd_gpu>)
#endif  // MXNET_USE_CUDA
    .set_attr<mxnet::FCompute>("FCompute<cpu>", mxnet::op::TVMBinaryCompute<func_vadd_cpu>)
    .set_attr<nnvm::FGradient>("FGradient", ElemwiseGradUseNone{"_backward_contrib_tvm_vadd"});

NNVM_REGISTER_OP(_backward_contrib_tvm_vadd)
    .set_num_inputs(1)
    .set_num_outputs(2)
    .set_attr<nnvm::TIsBackward>("TIsBackward", true)
#if MXNET_USE_CUDA
    .set_attr<mxnet::FCompute>("FCompute<gpu>",
                               mxnet::op::TVMBinaryBackwardComputeUseNone<func_bakcward_vadd_gpu>)
#endif  // MXNET_USE_CUDA
    .set_attr<mxnet::FCompute>("FCompute<cpu>",
                               mxnet::op::TVMBinaryBackwardComputeUseNone<func_bakcward_vadd_cpu>);
```

## Handle Different DTypes

https://docs.tvm.ai/tutorials/language/intrin_math.html#add-your-own-intrinsic

In [ ]:
def compute_xxx(dtype, ndim):
    A = tvm.placeholder([tvm.var() for _ in range(ndim)], name='A', dtype=dtype)
    B = tvm.placeholder([tvm.var() for _ in range(ndim)], name='B', dtype=dtype)
    C = tvm.compute([tvm.var() for _ in range(ndim)],
                    lambda *index: tvm.if_then_else(A.dtype.startswith("float"), A[index] - B[index], A[index] + B[index]),
                    name='C')
    s = tvm.create_schedule(C.op)
    return s, A, B, C

In [ ]:
s, A, B, C = compute_xxx("float32", 1)
tvm.lower(s, [A, B, C], simple_mode=True)

In [ ]:
if xxx:
    C = aaa
else:
    C = bbb

In [ ]:
@defop(name="vadd", target="cpu", auto_broadcast=True,
       dtype=["float32", "float64"], ndim=[5])
def vadd1(dtype, ndim):
    s, A, B, C = compute_add(dtype, ndim)
    axes = [axis for axis in C.op.axis]
    fused = s[C].fuse(*axes)
    s[C].parallel(fused)

    return s, [A, B, C]


@defop(name="vadd", target="cpu", auto_broadcast=True,
       dtype=["float16"], ndim=[5])
def vadd2(dtype, ndim):
    s, A, B, C = compute_add_f16(dtype, ndim)
    axes = [axis for axis in C.op.axis]
    fused = s[C].fuse(*axes)
    s[C].parallel(fused)

    return s, [A, B, C]

## Python frontend & unittest

......